In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
from langmem import create_memory_manager
from pydantic import BaseModel
from langchain_openai.chat_models import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

class Person(BaseModel):
    """Store a person's name, role, and preferences."""
    name: str
    role: str
    preferences: list[str] | None = None


manager = create_memory_manager(
    llm,
    schemas=[Person],
    instructions="Extract people's names, roles, and any mentioned preferences.",
    enable_inserts=True,
    enable_updates=True,
    enable_deletes=True,
)

In [ ]:
conversation = [
    {
        "role": "user",
        "content": (
            "John is a senior developer who loves coffee. "
            "Alice is a junior developer who hates coffee."
        )
    }
]
memories = manager.invoke({"messages": conversation})
print(memories)

In [ ]:
try:
    conversation_no_extraction = [
        {
            "role": "user",
            "content": (
                "Today it rained for two hours, and then the sun came out."
            )
        }
    ]

    memories_no_extraction = manager.invoke({"messages": conversation_no_extraction})
    print(memories_no_extraction)
except Exception as e:
    print(e)

In [ ]:
from langmem import create_memory_store_manager

namespace=("memories",)

memory_manager = create_memory_store_manager(
    llm,
    namespace=namespace,
    instructions="Only save information related about food the user likes"
)
try:
    memory_manager.invoke({"messages": ["I like dogs. My dog's name is Fido."]})
except Exception as e:
    print("Exception:", e)

In [ ]:
from langmem import ReflectionExecutor

executor = ReflectionExecutor(memory_manager)

In [ ]:
from langgraph.store.memory import InMemoryStore
from langgraph.func import entrypoint

store = InMemoryStore(
    index={
        "dims": 1536,
        "embed": "openai:text-embedding-3-small",
    }
)

@entrypoint(store=store)
async def chat(message: str):
    response = llm.invoke(message)

    to_process = {"messages": [{"role": "user", "content": message}] + [response]}
    await memory_manager.ainvoke(to_process)
    # executor.submit(to_process, after_seconds=1)
    return response.content

In [ ]:
response = await chat.ainvoke(
    "I like to eat Pizza",
)

In [ ]:
try:
    response = await chat.ainvoke(
        "I like dogs. My dog's name is Fido.",
    )
except Exception as e:
    print("Exception:", e)

In [ ]:
store.search(namespace)

### Better approach - Tools!

In [ ]:
from langgraph.store.memory import InMemoryStore
from langgraph.prebuilt import create_react_agent
from langmem import create_manage_memory_tool, create_search_memory_tool


store = InMemoryStore(
    index={
        "dims": 1536,
        "embed": "openai:text-embedding-3-small",
    }
)


In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

tools=[
    create_manage_memory_tool(namespace=("memories", "{user_id}"), store=store),
    create_search_memory_tool(namespace=("memories", "{user_id}"), store=store),
]

In [ ]:
app = create_react_agent(llm, tools=tools)

In [ ]:
app.invoke({"messages": [{"role": "user", "content": "hi!"}]}, config={"configurable": {"user_id": "alice"}})

In [ ]:
app.invoke({"messages": [{"role": "user", "content": "what do you know about me?"}]}, config={"configurable": {"user_id": "alice"}})

In [ ]:
app.invoke({"messages": [{"role": "user", "content": "I love spaghetti"}]}, config={"configurable": {"user_id": "alice"}})

In [ ]:
app.invoke({"messages": [{"role": "user", "content": "what do you know about me?"}]}, config={"configurable": {"user_id": "alice"}})

In [ ]:
app.invoke({"messages": [{"role": "user", "content": "what do you know about me?"}]}, config={"configurable": {"user_id": "max"}})

### Prededual Memory: System Instructions

In [22]:
from langmem import create_prompt_optimizer

optimizer = create_prompt_optimizer(
    llm,
    kind="metaprompt",
    config={"max_reflection_steps": 3}
)

In [23]:
prompt = "You are a helpful assistant."
trajectory = [
    {"role": "user", "content": "Explain inheritance in Python"},
    {"role": "assistant", "content": "Here's a detailed theoretical explanation..."},
    {"role": "user", "content": "Show me a practical example instead"},
]
optimized = optimizer.invoke({
    "trajectories": [(trajectory, {"user_score": 0})],
    "prompt": prompt
})
print(optimized)

You are a helpful assistant. When asked for explanations, prioritize providing practical examples, especially in programming contexts like Python. If the user requests an explanation, confirm whether they prefer a theoretical overview or a practical example.
